# RDP Generic CFS Bulk File Workflow - Remove Step 2

### Importing libararies

In [16]:
import os
import sys
import requests 
import json
from dotenv import dotenv_values
config = dotenv_values(".env")

### Set RDP credentials and Initial Parameters

In [17]:
username = config['RDP_USERNAME'] #Or replace with your RDP Machine-ID
password = config['RDP_PASSWORD'] #Or replace with your RDP Password
clientId = config['RDP_APP_KEY'] #Or replace with your RDP APP Key

RDP_HOST= 'https://api.refinitiv.com'
acccess_token = None
refresh_token = None
expires_in = 0

## <a id="rdp_workflow"></a>RDP APIs Application Workflow

### Step 1: Authentication with RDP APIs

Refinitiv Data Platform entitlement check is based on OAuth 2.0 specification. The first step of an application workflow is to get a token from RDP Auth Service, which will allow access to the protected resource, i.e. data REST API. 

The API requires the following access credential information:
- Username: The username. 
- Password: Password associated with the username. 
- Client ID: This is also known as ```AppKey```, and it is generated using an App key Generator. This unique identifier is defined for the user or application and is deemed confidential (not shared between users). The client_id parameter can be passed in the request body or as an “Authorization” request header that is encoded as base64.
- Grant Type ```password```: This is for initial authentication request. An application does not have any token, so it requests new tokens using username/password combination.

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/token HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Content-Length: XXX

username=RDP_USERNAME
&password=RDP_PASSWORD
&client_id=RDP_APP_KEY
&grant_type=password
&takeExclusiveSignOnControl=true
&scope=trapi
```

Once the authentication success, the function gets the RDP Auth service response message and keeps the following RDP token information in the variables.
- **access_token**: The token used to invoke REST data API calls as described above. The application must keep this credential for further RDP APIs requests.
- **refresh_token**: Refresh token to be used for obtaining an updated access token before expiration. The application must keep this credential for access token renewal.
- **expires_in**: Access token validity time in seconds.

Next, after the application received the Access Token (and authorization token) from RDP Auth Service, all subsequent REST API calls will use this token to get the data. Please find more detail regarding RDP APIs workflow in the following resources:
- [RDP APIs: Introduction to the Request-Response API](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api) page.
- [RDP APIs: Authorization - All about tokens](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens) page.

In [18]:
#step 1 - get RDP Access Token from RDP

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/token'
payload = f'grant_type=password&username={username}&client_id={clientId}&password={password}&takeExclusiveSignOnControl=True&scope=trapi'
try:
    response = requests.post(auth_url, 
                             headers = {'Content-Type':'application/x-www-form-urlencoded'}, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Authentication success')
    access_token = response.json()['access_token']
    refresh_token = response.json()['refresh_token']
    expires_in = int(response.json()['expires_in'])

if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Authentication success


## <a id="rdp_get_data"></a>Requesting Data from RDP APIs

That brings us to requesting the RDP APIs data. All subsequent REST API calls use the Access Token via the *Authorization* HTTP request message header as shown below to get the data. 
- Header: 
    * Authorization = ```Bearer <RDP Access Token>```

Please notice *the space* between the ```Bearer``` and ```RDP Access Token``` values.

The application then creates a request message in a JSON message format or URL query parameter based on the interested service and sends it as an HTTP request message to the Service Endpoint. Developers can get RDP APIs the Service Endpoint, HTTP operations, and parameters from Refinitiv Data Platform's [API Playground page](https://api.refinitiv.com/) - which is an interactive documentation site developers can access once they have a valid Refinitiv Data Platform account.

## <a id="rdp_get_esg_bulk"></a>Requesting Bulk Data

### Step 2: Listing the FileSets using the Bucket Name and Package ID

To request the CFS Bulk data, the first step is to send an HTTP ```GET``` request to the RDP ```/file-store/v1/file-sets?bucket={bucket-name}&packageId={packageId}``` endpoint to list all FileSets under the input ```bucket-name``` and ```packageId```.

The HTTP Request structure is as follows:

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/file-sets?bucket={bucket-name}&packageId={packageId} HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

The example bucket names and package Ids for RDP content set are as follows:

|              Content             |                  Bucket Name                 |              Example of Package ID             |
|:--------------------------------:|:-------------------------------------------:|:---------------------------------------------:|
| Financial Markets Reference Data | bulk-FMRD                                    | 4d48-d7ff-edcc3d38-8243-a4f7517962b8           |
| Symbology                        | bulk-Symbology                               | 4c80-73a0-fcef949b-bfde-2b9b8117cfb0           |
| ESG                              | bulk-ESG                                     | 4288-ebb6-93372235-acb2-89882a826af1           |
| ESG - Point in Time              | bulk-ESG                                     | 4173-aec7-8a0b0ac9-96f9-48e83ddbd2ad           |
| Tick History                     | TICKHISTORY_VBD_NO_EMBARGO                   | 4c01-ab9e-db594a31-a8f5-5b7852ec4638           |
| Green Revenue                    | bulk-GreenRevenue                            | Summary: 4e94-6d63-fea034dc-90e2-de33895bd4e9  |
| Green Revenue                    | bulk-GreenRevenue                            | Standard: 4316-d43b-81c40763-8e6a-0dbec8162ab1 |
| Starmine                         | STARMINE_PREDICTIVE_ANALYTICS_SMARTECON_LIVE | 40d4-1404-58533484-afe8-718650a4e0d4|       

**Note**: The bucket name is *case-insensitive*.

If you cannot find the bucket name or package Id for your interested content set, please contact your LSEG representative.

Next, set a bucket name to ```bucket_name``` and package Id to ```packageId``` variables  below like to following statement:

``` Python
bucket_name = 'bulk-Symbology'
packageId = '4c80-73a0-fcef949b-bfde-2b9b8117cfb0'
```

This notebook uses *bulk-ESG* bucket and *4288-ebb6-93372235-acb2-89882a826af1* packageId as an example.

In [19]:
# set Bucket Name, this notebook use bulk-ESG as an example

bucket_name = 'bulk-ESG'

# pick the packageId you need and set to the packageId variable
#packageId = response.json()['value'][0]['packageId']
packageId = '4288-ebb6-93372235-acb2-89882a826af1'

In [20]:
#step 2 - list FileSets from bucket name and package Id

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket={bucket_name}&packageId={packageId}'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive FileSets list from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive FileSets list from RDP APIs


Example of the first entry of package IDs, the pacakgeId is the ```packageId``` field.

In [21]:
print(json.dumps(response.json()['value'][0], sort_keys=True, indent=2, separators=(',', ':')))

{
  "attributes":[
    {
      "name":"ContentType",
      "value":"ESG Raw Full B"
    }
  ],
  "availableFrom":"2023-12-03T17:30:24Z",
  "availableTo":"2023-12-17T17:30:24Z",
  "bucketName":"bulk-ESG",
  "contentFrom":"1970-01-01T00:00:00Z",
  "contentTo":"2023-12-03T16:05:00Z",
  "created":"2023-12-03T17:30:24Z",
  "files":[
    "4b8b-03e4-018e3fdf-9ce9-013c70726edd",
    "4ff8-d5fa-deecba75-9692-d833271961e7"
  ],
  "id":"400c-3c99-69756558-a4d0-2fe9ff63be59",
  "modified":"2023-12-03T17:36:11Z",
  "name":"Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Jsonl-Init-2023-12-03T16:11:31.648Z",
  "numFiles":2,
  "packageId":"4288-ebb6-93372235-acb2-89882a826af1",
  "status":"READY"
}


The File ID is in the files array.

In [23]:
# try just one file
file_id = response.json()['value'][0]['files'][1]
print(file_id)

4ff8-d5fa-deecba75-9692-d833271961e7


### Step 2.5: Listing the FileSets using the Bucket Name - Paging

By default, the ```/file-store/v1/file-sets``` endpoint always returns 25 results per request. You can adjust the number of return results via the ```pageSize``` query parameter, the maximum number is **100**.

```HTTP
GET /file-store/v1/file-sets?bucket={bucket-name}&packageId={packageId}&pageSize={number}, HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```
Let's try with ```pageSize=2``` as an example.

In [24]:
#step 2.5 - list FileSets from bucket name and package Id - with pageSize 2

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket={bucket_name}&packageId={packageId}&pageSize=2'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive list Package IDs from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive list Package IDs from RDP APIs


In [25]:
print(json.dumps(response.json(), sort_keys=True, indent=2, separators=(',', ':')))

{
  "@nextLink":"/file-store/v1/file-sets?bucket=bulk-ESG&packageId=4288-ebb6-93372235-acb2-89882a826af1&pageSize=2&skipToken=ZmlsZXNldElkPTQxMzItMTAyNy01YTJjYjMwMy1hYzk0LWVmOGY5OWExNzU5ZA",
  "value":[
    {
      "attributes":[
        {
          "name":"ContentType",
          "value":"ESG Raw Full B"
        }
      ],
      "availableFrom":"2023-12-03T17:30:24Z",
      "availableTo":"2023-12-17T17:30:24Z",
      "bucketName":"bulk-ESG",
      "contentFrom":"1970-01-01T00:00:00Z",
      "contentTo":"2023-12-03T16:05:00Z",
      "created":"2023-12-03T17:30:24Z",
      "files":[
        "4b8b-03e4-018e3fdf-9ce9-013c70726edd",
        "4ff8-d5fa-deecba75-9692-d833271961e7"
      ],
      "id":"400c-3c99-69756558-a4d0-2fe9ff63be59",
      "modified":"2023-12-03T17:36:11Z",
      "name":"Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Jsonl-Init-2023-12-03T16:11:31.648Z",
      "numFiles":2,
      "packageId":"4288-ebb6-93372235-acb2-89882a826af1",
      "status":"READY"
    },
    {
      "at

Based on the response data above, you see the API returns **2 entries** per request as set via ```pageSize=2``` parameter. 

The ```@nextLink``` node contains the URL for requesting the next page of query as follows:

```HTTP
GET {@nextLink URL}, HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

Example:

In [26]:
if '@nextLink' in response.json():
    next_link = response.json()['@nextLink']
    #step 2.5 - list Package IDs from bucket name - with pageSize 2 - navigate to next link

    CFS_url = f'{RDP_HOST}{next_link}'
    
    try:
        response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
    except requests.exceptions.RequestException as exp:
        print(f'Caught exception: {exp}')
    
    if response.status_code == 200:  # HTTP Status 'OK'
        print('Receive list Package IDs from RDP APIs')
    else:
        print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
        print(f'Text: {response.text}')

Receive list Package IDs from RDP APIs


In [27]:
print(json.dumps(response.json(), sort_keys=True, indent=2, separators=(',', ':')))

{
  "@nextLink":"/file-store/v1/file-sets?bucket=bulk-ESG&skipToken=ZmlsZXNldElkPTQxYTUtYmY3Zi1jMzY2ZjFlNy1hYWExLTM1ODBmYzA1ZmZlZQ&packageId=4288-ebb6-93372235-acb2-89882a826af1&pageSize=2",
  "value":[
    {
      "attributes":[
        {
          "name":"ContentType",
          "value":"ESG Raw Full B"
        }
      ],
      "availableFrom":"2023-11-26T17:25:26Z",
      "availableTo":"2023-12-10T17:25:26Z",
      "bucketName":"bulk-ESG",
      "contentFrom":"2023-11-19T16:05:00Z",
      "contentTo":"2023-11-26T16:05:00Z",
      "created":"2023-11-26T17:25:26Z",
      "files":[
        "4ac3-ad9c-35a068fd-9487-09b2f3279c03"
      ],
      "id":"4142-951a-556014c5-a90d-d0b163dca985",
      "modified":"2023-11-26T17:35:27Z",
      "name":"Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Jsonl-Delta-2023-11-26T16:50:12.972Z",
      "numFiles":1,
      "packageId":"4288-ebb6-93372235-acb2-89882a826af1",
      "status":"READY"
    },
    {
      "attributes":[
        {
          "name":"Content

Then you can continue to send requests to URL in ```@nextLink``` node to get the next page results.

### Step 3: Get the Bulk file URL on AWS S3

The next step is getting the file URL on Amazon AWS S3 service with the RDP ```/file-store/v1/files/{file ID}/stream``` endpoint.

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/files/{fileId}/stream?doNotRedirect=true HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

In [28]:
#step 3 - get file URL from file id

FileID_url = f'{RDP_HOST}/file-store/v1/files/{file_id}/stream?doNotRedirect=true'

try:
    response = requests.get(FileID_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File URL from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File URL from RDP APIs


The File URL is in the ```url``` attribute of the response message.

In [30]:
file_url = response.json()['url']
print(file_url)

https://a206464-bulk-esg.s3.amazonaws.com/Bulk-ESG-Global-Raw-Full-SchemeB-v1/2023/12/03/Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Init-2023-12-03T16%3A11%3A31.648Z-part0.jsonl.gz?x-request-Id=2db02a8f-b5b1-4744-a02d-4aed64994cd0&x-package-id=4288-ebb6-93372235-acb2-89882a826af1&x-client-app-id=b4842f3904fb4a1fa18234796368799086c63541&x-file-name=Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Init-2023-12-03T16%3A11%3A31.648Z-part0.jsonl.gz&x-fileset-id=400c-3c99-69756558-a4d0-2fe9ff63be59&x-bucket-name=bulk-ESG&x-uuid=GESG1-178570&x-file-Id=4ff8-d5fa-deecba75-9692-d833271961e7&x-fileset-name=Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Jsonl-Init-2023-12-03T16%3A11%3A31.648Z&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjED4aCXVzLWVhc3QtMSJIMEYCIQDZ2Y5TM9KUGHf0Ul0yYuYxBOLx7wjtk1CiEokPyJet2QIhAKfJcDN9NulXAUo7NywBLPtIj%2Fndng0%2FBLCRY5OyUzDKKqMCCKf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMNjQyMTU3MTgxMzI2IgzXp%2F5jW7amdZtPCGkq9wHiE3fwHqcjpZpDYxVIuJifFeZA8Sl5i7hibchGWWxy

### Step 4: Downloading the file

Based on the S3 ```file_url``` above, the actual file name is *Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Init-2023-12-03T16_11_31.648Z-part0.jsonl.gz*. So you need to replace the escape character ```%3A``` with ```_``` (underscore) character.

**Note**: If you cannot download the file, please wait for a while and then retry download the file from the URL. Please do not flush the download requests.

In [31]:
#step 4 - Downlaod file
import polling2

zipfilename = file_url.split("?")[0].split("/")[-1].replace("%3A","_")
print(f'Downloading File {zipfilename} ...')

def test_result(response):
    return response.status_code == 200

try:
    response = polling2.poll(lambda: requests.get(file_url), 
                            step = 10,
                            poll_forever = True,
                            check_success= test_result)
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File Successfully')
    open(zipfilename, 'wb').write(response.content)
    print(f'{zipfilename} Saved')
else:
    print(f'RDP APIs: Request file failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File Successfully
Bulk-ESG-Global-Raw-Full-SchemeB-v1-Gov-Init-2023-12-03T16_11_31.648Z-part0.jsonl.gz Saved


Now you get the CFS Bulk file that you can extract and read the file.

That is all for the RDP CFS Bulk File workflow.

### Step 5: Refresh Token with RDP APIs

Before the session expires (based on the ```expires_in``` parameter, in seconds) , an application needs to send a Refresh Grant request message to RDP Authentication service to get a new access token before further request data from the platform.

The API requires the following access credential information:
- Refresh Token: The current Refresh Token value from the previous RDP Authentication call
- Client ID: This is also known as ```AppKey```, and it is generated using an App key Generator. This unique identifier is defined for the user or application and is deemed confidential (not shared between users). The client_id parameter can be passed in the request body or as an “Authorization” request header that is encoded as base64.
- Grant Type ```refresh_token```: This is for getting a new Access Token. 

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/token HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Content-Length: XXX

refresh_token={current_refresh_token}
&grant_type=refresh_token
&client_id=RDP_APP_KEY
```

Once the authentication success, the function gets **access_token**, **refresh_token**, and **expires_in** from the RDP Auth service response message the same as the previous RDP Authentication call. An application must keep those value for the next Refresh Token call.

#### Caution: API Limit

The RDP Authentication service has the API limit described on the [RDP APIs: Limitations and Guidelines for the RDP Authentication Service](https://developers.lseg.com/en/article-catalog/article/limitations-and-guidelines-for-the-rdp-authentication-service) article.  If the application flushes the authentication request messages (both ```password``` and ```refresh_token``` grant_type) beyond the limit, the account will be blocked by the API Gateway. 

In [32]:
#step 5 - Refreshing Token

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/token'
payload = f'grant_type=refresh_token&client_id={clientId}&refresh_token={refresh_token}'
try:
    response = requests.post(auth_url, 
                             headers = {'Content-Type':'application/x-www-form-urlencoded'}, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Refresh Token success')
    access_token = response.json()['access_token']
    refresh_token = response.json()['refresh_token']
    expires_in = int(response.json()['expires_in'])

if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Refresh Token success


### Step 6: Revoke Token to ending the session.

This revocation mechanism allows an application to invalidate its tokens if the end-user logs out, changes identity, or exits the respective application. Notifying the authorization server that the token is no longer needed allows the authorization server to clean up data associated with that token (e.g., session data) and the underlying authorization grant.

The API requires the following HTTP Header and Credential parameter information:
- Header: 
    * Authorization = ```Basic <App Key in Base64 format>```

Please notice *the space* between the ```Basic``` and ```App Key in Base64 format``` values.
- Body parameter
    * token: The current ```Access Token``` value from the previous RDP Authentication call

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/revoke HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Authorization: Basic <App Key in Base64>
Content-Length: XXX

token={current_Access_token}
```

In [33]:
#step 6 - Revoking Token

import base64

clientId_bytes = clientId.encode('ascii')
base64_bytes = base64.b64encode(clientId_bytes)
clientId_base64 = base64_bytes.decode('ascii')

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/revoke'
payload = f'token={access_token}'
try:
    response = requests.post(auth_url, 
                             headers = {
                                 'Content-Type':'application/x-www-form-urlencoded',
                                 'Authorization': f'Basic {clientId_base64}'
                             }, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Revoke Token success')
if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Revoke Token success


That is all for the RDP CFS Bulk file workflow.

## Next Steps

You may interested in the following resources for more detail about the CFS data usage:
- [Find environmental footprint of your bond portfolio](https://developers.lseg.com/en/article-catalog/article/Environmental_footprint_of_bond_portfolio) article
- [RDP APIs Green Revenues CFS Bulk file Workflow](https://github.com/LSEG-API-Samples/Example.RDP.Python.GreenRevenuesBulk) - a dedicate Green Revenue CFS workflow
- [RDP APIs ESG CFS Bulk file Workflow](https://github.com/LSEG-API-Samples/Example.RDP.Python.ESG.PointinTimeBulk) - a dedicate ESG CFS workflow

And much more on the [Developer Portal](https://developers.lseg.com/en) website.

## <a id="references"></a>References

That brings me to the end of my generic CFS Bulk file workflow project. For further details, please check out the following resources:

* [Refinitiv Data Platform APIs page](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) on the [Refinitiv Developer Community](https://developers.lseg.com/) website.
* [Refinitiv Data Platform APIs Playground page](https://apidocs.refinitiv.com/Apps/ApiDocs).
* [Refinitiv Data Platform APIs: Introduction to the Request-Response API](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api).
* [Refinitiv Data Platform APIs: Authorization - All about tokens](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens).
* [Limitations and Guidelines for the RDP Authentication Service](https://developers.lseg.com/en/article-catalog/article/limitations-and-guidelines-for-the-rdp-authentication-service) article.
* [Getting Started with Refinitiv Data Platform](https://developers.lseg.com/en/article-catalog/article/getting-start-with-refinitiv-data-platform) article.
* [CFS API User Guide](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#cfs-api-user-guide).


For any questions related to Refinitiv Data Platform APIs, please use the [RDP APIs Forum](https://community.developers.refinitiv.com/spaces/231/index.html) on the [Developers Community Q&A page](https://community.developers.refinitiv.com/).